# GIS 5571 - Lab 1

## Preamble

In [1]:
# Import required packages
import arcpy
#import fiona
import geopandas as gpd
import gdal
import io
import json
import pandas as pd
import requests
import shapefile
import time
import zipfile as zip

# Set workspace as project geodatabase
arcpy.env.workspace = r'F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb'
arcpy.env.workspace

'F:\\GIS5571\\Lab1\\Lab1_arc\\Lab1_arc.gdb'

## Minnesota Geospatial Commons

In [2]:
### Minnesota Geospatial Commons - Find all impaired wetlands within 10 miles of a MN airport

# Assign download link to variable 
MGC_airports = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_airports/shp_trans_airports.zip'
MGC_wetlands = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_pca/env_impaired_wetlands_2012/shp_env_impaired_wetlands_2012.zip'

# Put download link variables in list
MGC_content = [MGC_airports, MGC_wetlands]

# Generate empty list for download commands
MGC_dl = []

# Initiate for loop
for i in MGC_content:
    # Pull data from API using requests.get
    MGC_content_dl = requests.post(i)
    # Append content of pull to empty list
    MGC_dl.append(MGC_content_dl.content)

    # Initiation second for loop
for i in MGC_dl:
    # Cnvert content to bytes
    content_zip = zip.ZipFile(io.BytesIO(i))
    # Extract to destination folder
    content_zip.extractall(r'F:\GIS5571\Lab1\Lab1_arc\MGC_data')


In [3]:
# Projecting to a different geographic and projected coordinate system

arcpy.management.Project(r"F:\GIS5571\Lab1\Lab1_arc\MGC_data\Airports_in_Minnesota.shp", r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb\Airports_in_Minnesota_NAD", 'PROJCS["NAD_1983_NSRS2007_Contiguous_USA_Albers",GEOGCS["GCS_NAD_1983_NSRS2007",DATUM["D_NAD_1983_NSRS2007",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]', "'WGS_1984_(ITRF00)_To_NAD_1983 + NAD_1983_NSRS2007_To_WGS_1984_1'", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project(r"F:\GIS5571\Lab1\Lab1_arc\MGC_data\env_impaired_wetlands_2012_pts.shp", r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb\env_impaired_wetlands_2012_pts_NAD", 'PROJCS["NAD_1983_NSRS2007_Contiguous_USA_Albers",GEOGCS["GCS_NAD_1983_NSRS2007",DATUM["D_NAD_1983_NSRS2007",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]', "'WGS_1984_(ITRF00)_To_NAD_1983 + NAD_1983_NSRS2007_To_WGS_1984_1'", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'F:\\GIS5571\\Lab1\\Lab1_arc\\Lab1_arc.gdb\\env_impaired_wetlands_2012_pts_NAD'>

In [4]:
# Spatial join to find all impaired wetlands within 10 miles of an airport

with arcpy.EnvManager(scratchWorkspace=r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb", workspace=r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb"):
    arcpy.analysis.SpatialJoin("env_impaired_wetlands_2012_pts_NAD", "Airports_in_Minnesota_NAD", r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb\MNImpairedWetlandswithin10milesofAirport_py", "JOIN_ONE_TO_MANY", "KEEP_COMMON", 'AUID "AUID" true true false 10 Text 0 0,First,#,env_impaired_wetlands_2012_pts,AUID,0,10;WATER_NAME "WATER_NAME" true true false 254 Text 0 0,First,#,env_impaired_wetlands_2012_pts,WATER_NAME,0,254;LOCATION "LOCATION" true true false 254 Text 0 0,First,#,env_impaired_wetlands_2012_pts,LOCATION,0,254;CAT "CAT" true true false 254 Text 0 0,First,#,env_impaired_wetlands_2012_pts,CAT,0,254;CYCLE_LAST "CYCLE_LAST" true true false 254 Text 0 0,First,#,env_impaired_wetlands_2012_pts,CYCLE_LAST,0,254;Affected_U "Affected_U" true true false 254 Text 0 0,First,#,env_impaired_wetlands_2012_pts,Affected_U,0,254;NOPLN "NOPLN" true true false 50 Text 0 0,First,#,env_impaired_wetlands_2012_pts,NOPLN,0,50;APPROVED "APPROVED" true true false 254 Text 0 0,First,#,env_impaired_wetlands_2012_pts,APPROVED,0,254;NEEDSPLN "NEEDSPLN" true true false 254 Text 0 0,First,#,env_impaired_wetlands_2012_pts,NEEDSPLN,0,254;IMPAIR_PAR "IMPAIR_PAR" true true false 254 Text 0 0,First,#,env_impaired_wetlands_2012_pts,IMPAIR_PAR,0,254;NEW_2012 "NEW_2012" true true false 100 Text 0 0,First,#,env_impaired_wetlands_2012_pts,NEW_2012,0,100;B_P "B_P" true true false 8 Text 0 0,First,#,env_impaired_wetlands_2012_pts,B_P,0,8;M_IBI "M_IBI" true true false 8 Text 0 0,First,#,env_impaired_wetlands_2012_pts,M_IBI,0,8;DELISTED "DELISTED" true true false 50 Text 0 0,First,#,env_impaired_wetlands_2012_pts,DELISTED,0,50;HUC8 "HUC8" true true false 8 Text 0 0,First,#,env_impaired_wetlands_2012_pts,HUC8,0,8;HUC8_NAME "HUC8_NAME" true true false 100 Text 0 0,First,#,env_impaired_wetlands_2012_pts,HUC8_NAME,0,100;BASIN "BASIN" true true false 100 Text 0 0,First,#,env_impaired_wetlands_2012_pts,BASIN,0,100;WDWMO_NAME "WDWMO_NAME" true true false 50 Text 0 0,First,#,env_impaired_wetlands_2012_pts,WDWMO_NAME,0,50;WDWMO_TYPE "WDWMO_TYPE" true true false 15 Text 0 0,First,#,env_impaired_wetlands_2012_pts,WDWMO_TYPE,0,15;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,env_impaired_wetlands_2012_pts,Shape_Leng,-1,-1;ORIG_FID "ORIG_FID" true true false 9 Long 0 9,First,#,env_impaired_wetlands_2012_pts,ORIG_FID,-1,-1;LOCATIONID "LOCATIONID" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,LOCATIONID,0,254;COUNTY "COUNTY" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,COUNTY,0,254;CITY "CITY" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,CITY,0,254;FACILITYNA "FACILITYNA" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,FACILITYNA,0,254;OWNERSHIP "OWNERSHIP" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,OWNERSHIP,0,254;USE "USE" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,USE,0,254;OWNER "OWNER" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,OWNER,0,254;OWNER_ADDR "OWNER_ADDR" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,OWNER_ADDR,0,254;OWNER_CITY "OWNER_CITY" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,OWNER_CITY,0,254;OWNER_PHON "OWNER_PHON" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,OWNER_PHON,0,254;MANAGER "MANAGER" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,MANAGER,0,254;MANAGER_AD "MANAGER_AD" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,MANAGER_AD,0,254;MANAGER_CI "MANAGER_CI" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,MANAGER_CI,0,254;MANAGER_PH "MANAGER_PH" true true false 254 Text 0 0,First,#,Airports_in_Minnesota,MANAGER_PH,0,254;LATITUDE_D "LATITUDE_D" true true false 19 Double 0 0,First,#,Airports_in_Minnesota,LATITUDE_D,-1,-1;LONGITUDE_ "LONGITUDE_" true true false 19 Double 0 0,First,#,Airports_in_Minnesota,LONGITUDE_,-1,-1;ELEVATION "ELEVATION" true true false 19 Double 0 0,First,#,Airports_in_Minnesota,ELEVATION,-1,-1', "WITHIN_A_DISTANCE", "10 Miles", '')

In [5]:
# Display 'head' of new spatially joined attribute table
# Convert to accessible shapefile
arcpy.conversion.ExportFeatures("MNImpairedWetlandswithin10milesofAirport_py", r"F:\GIS5571\Lab1\Lab1_arc\shapefiles\MGC_head_py.shp", '', "NOT_USE_ALIAS", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,TARGET_FID,-1,-1;JOIN_FID "JOIN_FID" true true false 4 Long 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,JOIN_FID,-1,-1;AUID "AUID" true true false 10 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,AUID,0,10;WATER_NAME "WATER_NAME" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,WATER_NAME,0,254;LOCATION "LOCATION" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,LOCATION,0,254;CAT "CAT" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,CAT,0,254;CYCLE_LAST "CYCLE_LAST" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,CYCLE_LAST,0,254;Affected_U "Affected_U" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,Affected_U,0,254;NOPLN "NOPLN" true true false 50 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,NOPLN,0,50;APPROVED "APPROVED" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,APPROVED,0,254;NEEDSPLN "NEEDSPLN" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,NEEDSPLN,0,254;IMPAIR_PAR "IMPAIR_PAR" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,IMPAIR_PAR,0,254;NEW_2012 "NEW_2012" true true false 100 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,NEW_2012,0,100;B_P "B_P" true true false 8 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,B_P,0,8;M_IBI "M_IBI" true true false 8 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,M_IBI,0,8;DELISTED "DELISTED" true true false 50 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,DELISTED,0,50;HUC8 "HUC8" true true false 8 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,HUC8,0,8;HUC8_NAME "HUC8_NAME" true true false 100 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,HUC8_NAME,0,100;BASIN "BASIN" true true false 100 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,BASIN,0,100;WDWMO_NAME "WDWMO_NAME" true true false 50 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,WDWMO_NAME,0,50;WDWMO_TYPE "WDWMO_TYPE" true true false 15 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,WDWMO_TYPE,0,15;Shape_Leng "Shape_Leng" true true false 8 Double 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,Shape_Leng,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,ORIG_FID,-1,-1;LOCATIONID "LOCATIONID" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,LOCATIONID,0,254;COUNTY "COUNTY" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,COUNTY,0,254;CITY "CITY" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,CITY,0,254;FACILITYNA "FACILITYNA" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,FACILITYNA,0,254;OWNERSHIP "OWNERSHIP" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,OWNERSHIP,0,254;USE "USE" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,USE,0,254;OWNER "OWNER" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,OWNER,0,254;OWNER_ADDR "OWNER_ADDR" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,OWNER_ADDR,0,254;OWNER_CITY "OWNER_CITY" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,OWNER_CITY,0,254;OWNER_PHON "OWNER_PHON" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,OWNER_PHON,0,254;MANAGER "MANAGER" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,MANAGER,0,254;MANAGER_AD "MANAGER_AD" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,MANAGER_AD,0,254;MANAGER_CI "MANAGER_CI" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,MANAGER_CI,0,254;MANAGER_PH "MANAGER_PH" true true false 254 Text 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,MANAGER_PH,0,254;LATITUDE_D "LATITUDE_D" true true false 8 Double 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,LATITUDE_D,-1,-1;LONGITUDE_ "LONGITUDE_" true true false 8 Double 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,LONGITUDE_,-1,-1;ELEVATION "ELEVATION" true true false 8 Double 0 0,First,#,MNImpairedWetlandswithin10milesofAirport_py,ELEVATION,-1,-1', None)
# Assign shapefile file path to a variable
MGC_head = r'F:\GIS5571\Lab1\Lab1_arc\shapefiles\MGC_head_py.shp'
# Create list of desired fields
MGC_fields = ['Target_FID','WATER_NAME','LOCATION','COUNTY', 'LATITUDE_D', 'LONGITUDE_']
#Use SearchCursor to generate data frame of desired fields in feature class
MGC_head_df = pd.DataFrame((row for row in arcpy.da.SearchCursor(MGC_head, MGC_fields)))
# Use .head() to display desired number of rows in data frame
MGC_head_df.head(5)

,0,1,2,3,4,5
0,1,Pochardt Slough,Lyon County,LINCOLN,44.291633,-96.150311
1,2,Weltz Slough,Lyon County,LINCOLN,44.291633,-96.150311
2,4,Jones,Ramsey County,RAMSEY,45.076910,-93.204391
3,4,Jones,Ramsey County,RAMSEY,45.033299,-93.125221
4,4,Jones,Ramsey County,RAMSEY,44.942583,-93.107844


## Google Places

In [6]:
### Google Places - Find all cafes and Chinese restaurants within 1km of the Humphrey School

# Assign Google Places links to specific variables and put variables in list
    
GP_cafe = r'https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword=cafe&location=44.971430%2C-93.244791&radius=1000&type=cafe&key=MY_API_KEY'
GP_chinese = r'https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword=chinese%20restaurant&location=44.971430%2C-93.244791&radius=1000&type=restaurant&key=MY_API_KEY'
GP_links = [GP_cafe, GP_chinese]

# Create empty lists for both search terms (cafes and Chinese restauarnts), and create a list of those lists
cafe = []
chinese = []
GP_terms = [cafe, chinese]

# Create dictionary of parameters
params = {}

# Create simple index list
index = [0, 1]

# Initiate for loop, running once for each index value 
for i in index:
    # Pull data from the API
    res = requests.post(GP_links[i], params = params)
    # Load JSON content into dictionary
    results = json.loads(res.content)
    # Assign results to the values in the list of terms
    GP_terms[i].extend(results['results'])
    # Pause briefly
    time.sleep(2)
    # Continuing process while API results while there is another page
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.post(GP_links[i], params = params)
        GP_terms[i].extend(results['results'])
        time.sleep(2)

In [7]:
# Assign output file for cafes to a variable
cafe_csv = r'F:\GIS5571\Lab1\Lab1_arc\GP_data\cafes.txt'
# Assign output file for Chinese restaurants to a variable
chinese_csv = r'F:\GIS5571\Lab1\Lab1_arc\GP_data\chinese.txt'
# Add output variables to list
csv_filepaths = [cafe_csv, chinese_csv]

# Generate empty lists for each desired variable for each data set (name, latitude, longitude)
cafe_name = []
cafe_lat = []
cafe_lon = []
chinese_name = []
chinese_lat = []
chinese_lon = []

# For cafes, append each empty list with the JSON dictionary contents
for i in range(len(cafe)):
    cafe_i = cafe[i]
    try:
        cafe_name.append(cafe_i['name'])
    except:
        cafe_name.append('none')
    try:
        cafe_lat.append(cafe_i['geometry']['location']['lat'])
    except:
        cafe_lat.append('none')
    try:
        cafe_lon.append(cafe_i['geometry']['location']['lng'])
    except:
        cafe_lon.append('none')

# Generate dictionary for cafes
cafe_dict = {'name':cafe_name, 'lat':cafe_lat, 'lon':cafe_lon}

# Create data frame from truncated dictionary
cafe_df = pd.DataFrame(cafe_dict)

# Convert data frame to csv
cafe_df.to_csv(r'F:\GIS5571\Lab1\Lab1_arc\GP_data\cafes.csv', header = True, index = None, sep = ',', mode = 'w')

# Generate feature class from csv and append to map, projecting to WGS84
arcpy.management.XYTableToPoint(r"F:\GIS5571\Lab1\Lab1_arc\GP_data\cafes.csv", r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb\cafes_map_py", "lon", "lat", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

# Repeat process for Chinese restaurants
for i in range(len(chinese)):
    chinese_i = chinese[i]
    try:
        chinese_name.append(chinese_i['name'])
    except:
        chinese_name.append('none')
    try:
        chinese_lat.append(chinese_i['geometry']['location']['lat'])
    except:
        chinese_lat.append('none')
    try:
        chinese_lon.append(chinese_i['geometry']['location']['lng'])
    except:
        chinese_lon.append('none')
chinese_dict = {'name':chinese_name, 'lat':chinese_lat, 'lon':chinese_lon}
chinese_df = pd.DataFrame(chinese_dict)
chinese_df.to_csv(r'F:\GIS5571\Lab1\Lab1_arc\GP_data\chinese.csv', header = True, index = None, sep = ',', mode = 'w')
arcpy.management.XYTableToPoint(r"F:\GIS5571\Lab1\Lab1_arc\GP_data\chinese.csv", r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb\chinese_map_py", "lon", "lat", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

<Result 'F:\\GIS5571\\Lab1\\Lab1_arc\\Lab1_arc.gdb\\chinese_map_py'>

In [8]:
# Spatial join to find all cafes within 200m of a Chinese restauarnt - in chase you want a coffee after your chow mein
arcpy.analysis.SpatialJoin("cafes_map_py", "chinese_map_py", r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb\Cafeswithin200mofaChineseRestaurant_py", "JOIN_ONE_TO_MANY", "KEEP_COMMON", 'name "name" true true false 8000 Text 0 0,First,#,cafes_map_py,name,0,8000;lat "lat" true true false 8 Double 0 0,First,#,cafes_map_py,lat,-1,-1;lon "lon" true true false 8 Double 0 0,First,#,cafes_map_py,lon,-1,-1;name_1 "name" true true false 8000 Text 0 0,First,#,chinese_map_py,name,0,8000;lat_1 "lat" true true false 8 Double 0 0,First,#,chinese_map_py,lat,-1,-1;lon_1 "lon" true true false 8 Double 0 0,First,#,chinese_map_py,lon,-1,-1', "WITHIN_A_DISTANCE", "200 Meters", '')

<Result 'F:\\GIS5571\\Lab1\\Lab1_arc\\Lab1_arc.gdb\\Cafeswithin200mofaChineseRestaurant_py'>

In [9]:
# Display 'head' of new spatially joined attribute table
# Convert to accessible shapefile
arcpy.conversion.ExportFeatures("Cafeswithin200mofaChineseRestaurant_py", r"F:\GIS5571\Lab1\Lab1_arc\shapefiles\GP_head_py.shp", '', "NOT_USE_ALIAS", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,TARGET_FID,-1,-1;JOIN_FID "JOIN_FID" true true false 4 Long 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,JOIN_FID,-1,-1;name "name" true true false 8000 Text 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,name,0,8000;lat "lat" true true false 8 Double 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,lat,-1,-1;lon "lon" true true false 8 Double 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,lon,-1,-1;name_1 "name" true true false 8000 Text 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,name_1,0,8000;lat_1 "lat" true true false 8 Double 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,lat_1,-1,-1;lon_1 "lon" true true false 8 Double 0 0,First,#,Cafeswithin200mofaChineseRestaurant_py,lon_1,-1,-1', None)
# Assign shapefile file path to a variable
GP_head = r"F:\GIS5571\Lab1\Lab1_arc\shapefiles\GP_head_py.shp"
# Create list of desired fields
GP_fields = ['Target_FID','name','lat','lon']
#Use SearchCursor to generate data frame of desired fields in feature class
GP_head_df = pd.DataFrame((row for row in arcpy.da.SearchCursor(GP_head, GP_fields)))
# Use .head() to display desired number of rows in data frame
GP_head_df.head(5)

,0,1,2,3
0,2,Cafe Jote,44.970162,-93.246703
1,13,Hard Times Cafe,44.969637,-93.246185
2,14,Caribou Coffee,44.969591,-93.244597


## NDAWN

In [10]:
## NDAWN
# Assign NDAWN csv download links to specific variables and put variables in list
NDAWN_aug = 'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=174&station=142&station=138&station=161&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=172&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=152&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddmxt&variable=ddmxtt&variable=ddmnt&variable=ddmntt&variable=ddavt&variable=dddtr&variable=ddbst&variable=ddtst&variable=ddws&variable=ddmxws&variable=ddmxwst&variable=ddwd&variable=ddwdsd&variable=ddsr&variable=ddtpetp&variable=ddtpetjh&variable=ddr&variable=dddp&variable=ddwc&variable=ddmnwc&variable=ddmxt9&variable=ddmxtt9&variable=ddmnt9&variable=ddmntt9&variable=ddmxws10&variable=ddmxwst10&variable=ddwd10&variable=ddwdsd10&year=2022&ttype=daily&quick_pick=&begin_date=2022-08-18&end_date=2022-08-18'
NDAWN_feb = 'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=174&station=142&station=138&station=161&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=172&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=152&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddmxt&variable=ddmxtt&variable=ddmnt&variable=ddmntt&variable=ddavt&variable=dddtr&variable=ddbst&variable=ddtst&variable=ddws&variable=ddmxws&variable=ddmxwst&variable=ddwd&variable=ddwdsd&variable=ddsr&variable=ddtpetp&variable=ddtpetjh&variable=ddr&variable=dddp&variable=ddwc&variable=ddmnwc&variable=ddmxt9&variable=ddmxtt9&variable=ddmnt9&variable=ddmntt9&variable=ddmxws10&variable=ddmxwst10&variable=ddwd10&variable=ddwdsd10&year=2022&ttype=daily&quick_pick=&begin_date=2022-02-18&end_date=2022-02-18'

# Read csvs into data frame using read_csv, assigning header and skipping row containing units
aug_data_py = pd.read_csv(NDAWN_aug, header = 3, sep = ",", skiprows=[4])
feb_data_py = pd.read_csv(NDAWN_feb, header = 3, sep = ",", skiprows=[4])

# Export data frame as formatted csv
aug_data_py.to_csv(r'F:\GIS5571\Lab1\Lab1_arc\NDAWN_data\aug_data_py.csv', header = True, index = None, sep = ',', mode = 'w')
feb_data_py.to_csv(r'F:\GIS5571\Lab1\Lab1_arc\NDAWN_data\feb_data_py.csv', header = True, index = None, sep = ',', mode = 'w')

# Load csv data into Arc using 'Display XY Data'
arcpy.management.XYTableToPoint("aug_data_py.csv", r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb\NDAWN_aug_data_py", "Longitude", "Latitude", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
arcpy.management.XYTableToPoint("feb_data_py.csv", r"F:\GIS5571\Lab1\Lab1_arc\Lab1_arc.gdb\NDAWN_feb_data_py", "Longitude", "Latitude", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')


<Result 'F:\\GIS5571\\Lab1\\Lab1_arc\\Lab1_arc.gdb\\NDAWN_feb_data_py'>

In [11]:
# Spatial join august and february data sets
arcpy.analysis.SpatialJoin("NDAWN_feb_data_py", "NDAWN_aug_data_py", r"F:\GIS5571\Lab1\Lab1_arc\NDAWN_data_py", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Station_Name "Station Name" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Station_Name,0,8000,NDAWN_aug_data_py,Station_Name,0,8000;Latitude "Latitude" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Latitude,-1,-1,NDAWN_aug_data_py,Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Longitude,-1,-1,NDAWN_aug_data_py,Longitude,-1,-1;Elevation "Elevation" true true false 4 Long 0 0,First,#,NDAWN_feb_data_py,Elevation,-1,-1,NDAWN_aug_data_py,Elevation,-1,-1;Year "Year" true true false 4 Long 0 0,First,#,NDAWN_feb_data_py,Year,-1,-1,NDAWN_aug_data_py,Year,-1,-1;Month "Month" true true false 4 Long 0 0,First,#,NDAWN_feb_data_py,Month,-1,-1,NDAWN_aug_data_py,Month,-1,-1;Day "Day" true true false 4 Long 0 0,First,#,NDAWN_feb_data_py,Day,-1,-1,NDAWN_aug_data_py,Day,-1,-1;Max_Temp "Max Temp" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Max_Temp,-1,-1,NDAWN_aug_data_py,Max_Temp,-1,-1;Max_Temp_Flag "Max Temp Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Temp_Flag,0,8000,NDAWN_aug_data_py,Max_Temp_Flag,0,8000;Max_Temp_Time "Max Temp Time" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Temp_Time,0,8000,NDAWN_aug_data_py,Max_Temp_Time,0,8000;Max_Temp_Time_Flag "Max Temp Time Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Temp_Time_Flag,0,8000,NDAWN_aug_data_py,Max_Temp_Time_Flag,0,8000;Min_Temp "Min Temp" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Min_Temp,-1,-1,NDAWN_aug_data_py,Min_Temp,-1,-1;Min_Temp_Flag "Min Temp Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Min_Temp_Flag,0,8000,NDAWN_aug_data_py,Min_Temp_Flag,0,8000;Min_Temp_Time "Min Temp Time" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Min_Temp_Time,0,8000,NDAWN_aug_data_py,Min_Temp_Time,0,8000;Min_Temp_Time_Flag "Min Temp Time Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Min_Temp_Time_Flag,0,8000,NDAWN_aug_data_py,Min_Temp_Time_Flag,0,8000;Avg_Temp "Avg Temp" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Avg_Temp,-1,-1,NDAWN_aug_data_py,Avg_Temp,-1,-1;Avg_Temp_Flag "Avg Temp Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Avg_Temp_Flag,0,8000,NDAWN_aug_data_py,Avg_Temp_Flag,0,8000;Diurnal_Temp_Range "Diurnal Temp Range" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Diurnal_Temp_Range,-1,-1,NDAWN_aug_data_py,Diurnal_Temp_Range,-1,-1;Diurnal_Temp_Range_Flag "Diurnal Temp Range Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Diurnal_Temp_Range_Flag,0,8000,NDAWN_aug_data_py,Diurnal_Temp_Range_Flag,0,8000;Avg_Bare_Soil_Temp "Avg Bare Soil Temp" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Avg_Bare_Soil_Temp,-1,-1,NDAWN_aug_data_py,Avg_Bare_Soil_Temp,-1,-1;Avg_Bare_Soil_Temp_Flag "Avg Bare Soil Temp Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Avg_Bare_Soil_Temp_Flag,0,8000,NDAWN_aug_data_py,Avg_Bare_Soil_Temp_Flag,0,8000;Avg_Turf_Soil_Temp "Avg Turf Soil Temp" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Avg_Turf_Soil_Temp,-1,-1,NDAWN_aug_data_py,Avg_Turf_Soil_Temp,-1,-1;Avg_Turf_Soil_Temp_Flag "Avg Turf Soil Temp Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Avg_Turf_Soil_Temp_Flag,0,8000,NDAWN_aug_data_py,Avg_Turf_Soil_Temp_Flag,0,8000;Avg_Wind_Speed "Avg Wind Speed" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Speed,-1,-1,NDAWN_aug_data_py,Avg_Wind_Speed,-1,-1;Avg_Wind_Speed_Flag "Avg Wind Speed Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Speed_Flag,0,8000,NDAWN_aug_data_py,Avg_Wind_Speed_Flag,0,8000;Max_Wind_Speed "Max Wind Speed" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Max_Wind_Speed,-1,-1,NDAWN_aug_data_py,Max_Wind_Speed,-1,-1;Max_Wind_Speed_Flag "Max Wind Speed Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Wind_Speed_Flag,0,8000,NDAWN_aug_data_py,Max_Wind_Speed_Flag,0,8000;Max_Wind_Speed_Time "Max Wind Speed Time" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Wind_Speed_Time,0,8000,NDAWN_aug_data_py,Max_Wind_Speed_Time,0,8000;Max_Wind_Speed_Time_Flag "Max Wind Speed Time Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Wind_Speed_Time_Flag,0,8000,NDAWN_aug_data_py,Max_Wind_Speed_Time_Flag,0,8000;Avg_Wind_Dir "Avg Wind Dir" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Dir,-1,-1,NDAWN_aug_data_py,Avg_Wind_Dir,-1,-1;Avg_Wind_Dir_Flag "Avg Wind Dir Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Dir_Flag,0,8000,NDAWN_aug_data_py,Avg_Wind_Dir_Flag,0,8000;Avg_Wind_Dir_SD "Avg Wind Dir SD" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Dir_SD,-1,-1,NDAWN_aug_data_py,Avg_Wind_Dir_SD,-1,-1;Avg_Wind_Dir_SD_Flag "Avg Wind Dir SD Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Dir_SD_Flag,0,8000,NDAWN_aug_data_py,Avg_Wind_Dir_SD_Flag,0,8000;Total_Solar_Rad "Total Solar Rad" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Total_Solar_Rad,-1,-1,NDAWN_aug_data_py,Total_Solar_Rad,-1,-1;Total_Solar_Rad_Flag "Total Solar Rad Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Total_Solar_Rad_Flag,0,8000,NDAWN_aug_data_py,Total_Solar_Rad_Flag,0,8000;Penman_PET "Penman PET" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Penman_PET,-1,-1,NDAWN_aug_data_py,Penman_PET,-1,-1;Penman_PET_Flag "Penman PET Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Penman_PET_Flag,0,8000,NDAWN_aug_data_py,Penman_PET_Flag,0,8000;Jensen_Haise_PET "Jensen-Haise PET" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Jensen_Haise_PET,-1,-1,NDAWN_aug_data_py,Jensen_Haise_PET,-1,-1;Jensen_Haise_PET_Flag "Jensen-Haise PET Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Jensen_Haise_PET_Flag,0,8000,NDAWN_aug_data_py,Jensen_Haise_PET_Flag,0,8000;Rainfall "Rainfall" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Rainfall,0,8000,NDAWN_aug_data_py,Rainfall,-1,-1;Rainfall_Flag "Rainfall Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Rainfall_Flag,0,8000,NDAWN_aug_data_py,Rainfall_Flag,0,8000;Dew_Point "Dew Point" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Dew_Point,-1,-1,NDAWN_aug_data_py,Dew_Point,-1,-1;Dew_Point_Flag "Dew Point Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Dew_Point_Flag,0,8000,NDAWN_aug_data_py,Dew_Point_Flag,0,8000;Wind_Chill "Wind Chill" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Wind_Chill,-1,-1,NDAWN_aug_data_py,Wind_Chill,-1,-1;Wind_Chill_Flag "Wind Chill Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Wind_Chill_Flag,0,8000,NDAWN_aug_data_py,Wind_Chill_Flag,0,8000;Min_Wind_Chill "Min Wind Chill" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Min_Wind_Chill,-1,-1,NDAWN_aug_data_py,Min_Wind_Chill,-1,-1;Min_Wind_Chill_Flag "Min Wind Chill Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Min_Wind_Chill_Flag,0,8000,NDAWN_aug_data_py,Min_Wind_Chill_Flag,0,8000;Max_Temp_at_9_m "Max Temp at 9 m" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Max_Temp_at_9_m,-1,-1,NDAWN_aug_data_py,Max_Temp_at_9_m,-1,-1;Max_Temp_at_9_m_Flag "Max Temp at 9 m Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Temp_at_9_m_Flag,0,8000,NDAWN_aug_data_py,Max_Temp_at_9_m_Flag,0,8000;Max_Temp_Time_at_9_m "Max Temp Time at 9 m" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Temp_Time_at_9_m,0,8000,NDAWN_aug_data_py,Max_Temp_Time_at_9_m,0,8000;Max_Temp_Time_at_9_m_Flag "Max Temp Time at 9 m Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Temp_Time_at_9_m_Flag,0,8000,NDAWN_aug_data_py,Max_Temp_Time_at_9_m_Flag,0,8000;Min_Temp_at_9_m "Min Temp at 9 m" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Min_Temp_at_9_m,-1,-1,NDAWN_aug_data_py,Min_Temp_at_9_m,-1,-1;Min_Temp_at_9_m_Flag "Min Temp at 9 m Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Min_Temp_at_9_m_Flag,0,8000,NDAWN_aug_data_py,Min_Temp_at_9_m_Flag,0,8000;Min_Temp_Time_at_9_m "Min Temp Time at 9 m" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Min_Temp_Time_at_9_m,0,8000,NDAWN_aug_data_py,Min_Temp_Time_at_9_m,0,8000;Min_Temp_Time_at_9_m_Flag "Min Temp Time at 9 m Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Min_Temp_Time_at_9_m_Flag,0,8000,NDAWN_aug_data_py,Min_Temp_Time_at_9_m_Flag,0,8000;Max_Wind_Speed_at_10_m "Max Wind Speed at 10 m" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Max_Wind_Speed_at_10_m,-1,-1,NDAWN_aug_data_py,Max_Wind_Speed_at_10_m,-1,-1;Max_Wind_Speed_at_10_m_Flag "Max Wind Speed at 10 m Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Wind_Speed_at_10_m_Flag,0,8000,NDAWN_aug_data_py,Max_Wind_Speed_at_10_m_Flag,0,8000;Max_Wind_Speed_Time_at_10_m "Max Wind Speed Time at 10 m" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Wind_Speed_Time_at_10_m,0,8000,NDAWN_aug_data_py,Max_Wind_Speed_Time_at_10_m,0,8000;Max_Wind_Speed_Time_at_10_m_Flag "Max Wind Speed Time at 10 m Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Max_Wind_Speed_Time_at_10_m_Flag,0,8000,NDAWN_aug_data_py,Max_Wind_Speed_Time_at_10_m_Flag,0,8000;Avg_Wind_Dir_at_10_m "Avg Wind Dir at 10 m" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Dir_at_10_m,-1,-1,NDAWN_aug_data_py,Avg_Wind_Dir_at_10_m,-1,-1;Avg_Wind_Dir_at_10_m_Flag "Avg Wind Dir at 10 m Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Dir_at_10_m_Flag,0,8000,NDAWN_aug_data_py,Avg_Wind_Dir_at_10_m_Flag,0,8000;Avg_Wind_Dir_SD_at_10_m "Avg Wind Dir SD at 10 m" true true false 8 Double 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Dir_SD_at_10_m,-1,-1,NDAWN_aug_data_py,Avg_Wind_Dir_SD_at_10_m,-1,-1;Avg_Wind_Dir_SD_at_10_m_Flag "Avg Wind Dir SD at 10 m Flag" true true false 8000 Text 0 0,First,#,NDAWN_feb_data_py,Avg_Wind_Dir_SD_at_10_m_Flag,0,8000,NDAWN_aug_data_py,Avg_Wind_Dir_SD_at_10_m_Flag,0,8000', "INTERSECT", None, '')

<Result 'F:\\GIS5571\\Lab1\\Lab1_arc\\NDAWN_data_py.shp'>

In [12]:
# Assign shapefile file path to a variabe
NDAWN_head = r'F:\GIS5571\Lab1\Lab1_arc\NDAWN_data_py'
# Create list of desired fields
NDAWN_fields = ['Station_Na','Latitude','Longitude','Max_Temp', 'Min_Temp', 'Avg_Temp']
#Use SearchCursor to generate data frame of desired fields in feature class
NDAWN_head_df = pd.DataFrame((row for row in arcpy.da.SearchCursor(NDAWN_head, NDAWN_fields)))
# Use .head() to display desired number of rows in data frame
NDAWN_head_df.head(5)

,0,1,2,3,4,5
0,Ada,47.321190,-96.514060,20.863,-2.128,9.368
1,Adams,48.499880,-98.075880,16.160,-8.986,3.587
2,Alamo,48.546520,-103.471860,34.421,-8.500,12.961
3,Alvarado,48.245942,-97.021532,19.330,-3.046,8.142
4,Amidon,46.488440,-103.316290,35.956,6.026,20.991
